In [1]:
L = [0,7,9,1,5,3]
L.sort()
L

[0, 1, 3, 5, 7, 9]

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [3]:
L = [0,7,9,1,5,3]
sorted_indices = np.argsort(L)
print(f"sorted_indices = {sorted_indices}")
print(f"L[sorted_indices] = {np.array(L)[sorted_indices]}")

sorted_indices = [0 3 5 4 1 2]
L[sorted_indices] = [0 1 3 5 7 9]


# RNN or Sequential Models in General
Sequential models can deal with input of arbitrary length no problem. However, as one dives deeper into conceiving
a model, one would soon find out that the implementation details requires some design:

01. If we only design a normal sequential model, it will still **suffer from a similar issue** when we wanted to use vanilla NN, i.e. although the input can be now be of arbitrary length, some of the sequential models can only have fixed-length output.
  - In our case, i.e. **the output sequence length equals the input sequence length** [The sorted array's length should of course equal the original array's length], two models are particularly suited: **encoder-decoder** and **seq-to-seq** models.
  - Seq-to-seq models might
02. What kind of output should our sequential model produce?
03. What should the loss function look like?
04. Dataset.

## Dataset
If we take every array into consideration, how many are there?<br>
Well,

- for length-2 arrays, there are `10*9` of them
- for length-3 arrays, there are `10*9*8` of them
- ...
- for length-10 arrays, there are $10!$ of them

Let's calculate their sum using Python.

In [4]:
list(range(10,8))

[]

In [5]:
list(range(10,8,-1))

[10, 9]

In [6]:
from functools import reduce

In [7]:
reduce(lambda x, y: x*y, range(10,7,-1))

720

The following cell should give the right number.

In [8]:
sum([reduce(lambda x, y: x*y, range(10,10-length,-1))
     for length in range(2, 10+1)])

9864090

I just wanted to have an estimate of the number of arrays we can have. Now, how should we generate and store our data for training?

Let's first try with brute-force. Store the entire dataset into a dictionary with keys as any array and value as the sorted array.

In [9]:
from itertools import combinations, permutations

In [10]:
S = set(range(0, 9+1))

In [11]:
list(combinations(S, 2))

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (3, 8),
 (3, 9),
 (4, 5),
 (4, 6),
 (4, 7),
 (4, 8),
 (4, 9),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (6, 7),
 (6, 8),
 (6, 9),
 (7, 8),
 (7, 9),
 (8, 9)]

In [12]:
list(permutations([1, 2, 3]))

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [13]:
for i in [3.14, 2.7281, 123.501]:
    pass
i

123.501

In [14]:
# note that it returns a list although the input is a tuple
sorted((3,2,1))

[1, 2, 3]

In [15]:
%%time
# (key, value) of D will be of type (tuple, list)
D = {}
S = set(range(0, 9+1))
for length in range(2, 10+1):
    for c in itertools.combinations(S, length):
        c_sorted = sorted(c)
        for p in permutations(c):
            D[p] = c_sorted

NameError: name 'itertools' is not defined

Well, it took less time than I thought, even on an old laptop as Thinkpad X220 (Several seconds).

In [16]:
len(D)

0

In [17]:
len(D) == sum([reduce(lambda x, y: x*y, range(10,10-length,-1))
     for length in range(2, 10+1)])

False

To train on sequences of diff lengths, we must **batch sequences of identical lengths together**. This poses a new difficulty...

**Rmk**. Let's **use every single one of them as our training set** and **no test set**, because in this particular task, we care less about its generalization ability. If ever later on we care that, we can include arrays of a larger variety of numbers (e.g. all floating point numbers).

Since there are

- `10*9` length-2 arrays
- `10*9*8` length-3 arrays
- ...

We can set our batch size to be `10`, a common divisor of all the numbers above.

**Rmk**. I have had no prior experience dealing with batches of different lengths of sequences. I referred to [this stackexchange post](https://datascience.stackexchange.com/questions/26366/training-an-rnn-with-examples-of-different-lengths-in-keras).

In [18]:
A = [1,2,3,4,5,]
np.random.shuffle(A)
A

[2, 1, 3, 5, 4]

In [19]:
shuffled_lengths = list(range(2, 10+1))
np.random.shuffle(shuffled_lengths)
shuffled_lengths

[8, 10, 7, 9, 4, 2, 3, 6, 5]

In [20]:
np.empty((3,2))

array([[0.0e+000, 4.9e-324],
       [1.5e-323, 2.5e-323],
       [3.5e-323, 4.4e-323]])

In [21]:
n_classes = 10

In [22]:
X = np.empty((3,3,4))
X[1:, ...] = np.arange(3*4).reshape((3,4))
X

array([[[1.13682005e-313, 0.00000000e+000, 4.65641493e-310,
         0.00000000e+000],
        [4.65641493e-310, 0.00000000e+000, 4.65641493e-310,
         4.65641493e-310],
        [0.00000000e+000, 4.65641493e-310, 0.00000000e+000,
         0.00000000e+000]],

       [[0.00000000e+000, 1.00000000e+000, 2.00000000e+000,
         3.00000000e+000],
        [4.00000000e+000, 5.00000000e+000, 6.00000000e+000,
         7.00000000e+000],
        [8.00000000e+000, 9.00000000e+000, 1.00000000e+001,
         1.10000000e+001]],

       [[0.00000000e+000, 1.00000000e+000, 2.00000000e+000,
         3.00000000e+000],
        [4.00000000e+000, 5.00000000e+000, 6.00000000e+000,
         7.00000000e+000],
        [8.00000000e+000, 9.00000000e+000, 1.00000000e+001,
         1.10000000e+001]]])

In [23]:
def one_hot(array, depth=n_classes):
    """
    array is an ndarray/list of shape (None,)
    """
    return np.eye(depth)[array, :]

In [24]:
c_sorted = np.array([0, 4, 7 ,9])
one_hot(c_sorted)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [25]:
tf.one_hot(c_sorted, depth=n_classes)

<tf.Tensor: shape=(4, 10), dtype=float32, numpy=
array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)>

In [26]:
import math

def train_generator(batch_size=10):
    shuffled_lengths = list(range(2, 10+1))
    np.random.shuffle(shuffled_lengths)
    for length in shuffled_lengths:
        n_instances = reduce(lambda x, y: x*y, range(10,10-length,-1))
        X = np.empty((n_instances, length, 1))
        Y = np.empty((n_instances, length, n_classes))
        n_permutations = math.factorial(length)
        #n_combinations = n_instances // n_permutations
        for i, c in enumerate(combinations(S, length)):
            c_sorted = np.array(sorted(c))  # shape (length,)
            c_onehot = one_hot(c_sorted)    # shape (length, n_classes)
            Y[i*n_permutations : i*n_permutations + n_permutations, ...] = c_onehot
            for j, p in enumerate(permutations(c)):
                X[i*n_permutations : i*n_permutations + j, ...] = np.array(p)[..., np.newaxis]
        # Throw one batch after another to the model
        for k in range(n_instances // batch_size):
            yield X[k*batch_size: (k+1)*batch_size].astype(np.float32), Y[k*batch_size: (k+1)*batch_size]

## Seq-to-Seq Model
This is the seq-to-seq model in which the output sequence's length equals the input sequence length. We had better used **bidirectional RNNs**.

In [31]:
seq2seq_LSTM_model = keras.models.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(10,
        return_sequences=True, input_shape=[None, 1], dropout=0.2)),
    keras.layers.LSTM(10, return_sequences=True, dropout=0.2),
    #keras.layers.Bidirectional(keras.layers.LSTM(10,
    #    return_sequences=True, dropout=0.2)),
    keras.layers.TimeDistributed(keras.layers.Dense(n_classes, activation="softmax")),
])

#seq2seq_LSTM_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
seq2seq_LSTM_model.compile(loss="categorical_crossentropy",
                           optimizer="adam")

Before we jump into the potentially time-consuming training (via the `fit()` method), we can ask our model make a few predictions first to see if we implemented everything right.
01. With `dropout=0.2` in the first layer and predicting on `np.array([0,9,8,3]` produced
    ```
    TypeError: Input 'b' of 'MatMul' Op has type float32 that does not match type int64 of argument 'a'.
    ```
    And if we execute the cell again, we will get
    ```
    TypeError: 'NoneType' object is not callable
    ```
    Quite weird behaviour.
    - Actually, not `dropout=0.2`'s fault, the same error remains when we erase that input arg.
02. Once the input has been corrected to an ndarray of `dtype` equal to `float` and the ndarray shape to `n_batches, length (aka n_timesteps), 1`, prediction works no problem.

In [35]:
output = seq2seq_LSTM_model.predict(np.array([0.,9,8,3]).reshape((1,4,1)))
output.shape

(1, 4, 10)

In [36]:
output = seq2seq_LSTM_model.predict(np.array([0.,9,8,3]).reshape((1,4,1)), batch_size=1)
output.shape

(1, 4, 10)

In [39]:
output.shape

(1, 4, 10)

In [40]:
np.argmax(output[0], axis=1)

array([1, 1, 1, 9])

In [47]:
np.random.choice(range(0,9+1), 10, replace=False).astype(np.float32)

array([4., 0., 3., 2., 9., 8., 5., 1., 6., 7.], dtype=float32)

In [49]:
l = 4
bs = 7
X_new = np.empty((bs, l, 1))
for i in range(X_new.shape[0]):
    X_new[i] = np.random.choice(range(0,9+1), l, replace=False).astype(np.float32).reshape((-1, 1))
output = seq2seq_LSTM_model.predict(X_new)
output.shape

(7, 4, 10)

In [50]:
output

array([[[0.10139951, 0.10696197, 0.10170557, 0.10122366, 0.09184594,
         0.09352627, 0.10196288, 0.10176584, 0.10014071, 0.09946768],
        [0.10589212, 0.11321083, 0.09876852, 0.10716182, 0.08725755,
         0.08694492, 0.09820793, 0.09859796, 0.09694368, 0.10701466],
        [0.11091924, 0.11816201, 0.09596454, 0.11186716, 0.08430978,
         0.08170394, 0.09514006, 0.09433002, 0.0929394 , 0.11466388],
        [0.11462069, 0.12173522, 0.09384915, 0.11450148, 0.08297214,
         0.07806396, 0.09311385, 0.09090093, 0.08934669, 0.12089601]],

       [[0.10228409, 0.10743177, 0.09990551, 0.10408314, 0.09215908,
         0.09249651, 0.09901246, 0.10085864, 0.09895333, 0.10281549],
        [0.10832956, 0.11387819, 0.09661565, 0.1100869 , 0.08730229,
         0.08573652, 0.09561523, 0.09615359, 0.09502178, 0.11126029],
        [0.1102557 , 0.1184758 , 0.09541058, 0.11385911, 0.08402092,
         0.08064597, 0.09313145, 0.09470943, 0.09250716, 0.11698385],
        [0.11399283, 0.12

It's already quite long as a notebook, let's continue the training part in `./02_run01.ipynb`.